In [1]:
import pandas as pd

In [2]:
%store -r df

In [3]:
df.head()

,variables,industry,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
1,Inversion Es(millones€),"01 AGRICULTURA, GANADERÍA, CAZA Y SERV. RELACI...",NaN,267.75,210.97,120.20,NaN,1514.55,1.51,25499.03,...,0.0,2.41,580.73,1800.97,6257.97,1503.5,3.06,1.55,2.55,NaN
2,Inversion Es(millones€),02 SILVICULTURA Y EXPLOTACIÓN FORESTAL,NaN,NaN,NaN,0.12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,Inversion Es(millones€),03 PESCA Y ACUICULTURA,NaN,NaN,36.06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
4,Inversion Es(millones€),"05 EXTRACCIÓN DE ANTRACITA, HULLA Y LIGNITO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Inversion Es(millones€),06 EXTRACCIÓN DE CRUDO DE PETRÓLEO Y GAS NATURAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.50,NaN


In [5]:
df = df.transpose()

In [6]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,79,80,81,82,83,84,85,86,87,88
variables,Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€),...,Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€),Inversion Es(millones€)
industry,"01 AGRICULTURA, GANADERÍA, CAZA Y SERV. RELACI...",02 SILVICULTURA Y EXPLOTACIÓN FORESTAL,03 PESCA Y ACUICULTURA,"05 EXTRACCIÓN DE ANTRACITA, HULLA Y LIGNITO",06 EXTRACCIÓN DE CRUDO DE PETRÓLEO Y GAS NATURAL,07 EXTRACCIÓN DE MINERALES METÁLICOS,08 OTRAS INDUSTRIAS EXTRACTIVAS,09 ACTIVIDADES DE APOYO A LAS INDUSTRIAS EXTRA...,10 INDUSTRIA DE LA ALIMENTACIÓN,11 FABRICACIÓN DE BEBIDAS,...,"90 ACTIVIDADES DE CREACIÓN, ARTÍSTICAS Y ESPEC...","91 ACTIV. DE BIBLIOTECAS, ARCHIVOS, MUSEOS Y O...",92 ACTIVIDADES DE JUEGOS DE AZAR Y APUESTAS,"93 ACTIVIDADES DEPORTIVAS, RECREATIVAS Y DE EN...",94 ACTIVIDADES ASOCIATIVAS,"95 REPARACIÓN ORDENADORES, EFECTOS PERSONALES",96 OTROS SERVICIOS PERSONALES,97 ACTIVIDADES DE LOS HOGARES COMO EMPLEADORES,98 ACTIV. HOGARES COMO PRODUC. BIENES Y SERV,99 ACTIV. ORGANIZACIONES Y ORGANISMOS EXTRATER...
1993,NaN,NaN,NaN,NaN,NaN,NaN,235.6,NaN,NaN,NaN,...,NaN,NaN,NaN,1.8,NaN,NaN,88.35,NaN,NaN,NaN
1994,267.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995,210.97,NaN,36.06,NaN,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,NaN,180.3,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.drop(['variables'], axis=0)

,1,2,3,4,5,6,7,8,9,10,...,79,80,81,82,83,84,85,86,87,88
industry,"01 AGRICULTURA, GANADERÍA, CAZA Y SERV. RELACI...",02 SILVICULTURA Y EXPLOTACIÓN FORESTAL,03 PESCA Y ACUICULTURA,"05 EXTRACCIÓN DE ANTRACITA, HULLA Y LIGNITO",06 EXTRACCIÓN DE CRUDO DE PETRÓLEO Y GAS NATURAL,07 EXTRACCIÓN DE MINERALES METÁLICOS,08 OTRAS INDUSTRIAS EXTRACTIVAS,09 ACTIVIDADES DE APOYO A LAS INDUSTRIAS EXTRA...,10 INDUSTRIA DE LA ALIMENTACIÓN,11 FABRICACIÓN DE BEBIDAS,...,"90 ACTIVIDADES DE CREACIÓN, ARTÍSTICAS Y ESPEC...","91 ACTIV. DE BIBLIOTECAS, ARCHIVOS, MUSEOS Y O...",92 ACTIVIDADES DE JUEGOS DE AZAR Y APUESTAS,"93 ACTIVIDADES DEPORTIVAS, RECREATIVAS Y DE EN...",94 ACTIVIDADES ASOCIATIVAS,"95 REPARACIÓN ORDENADORES, EFECTOS PERSONALES",96 OTROS SERVICIOS PERSONALES,97 ACTIVIDADES DE LOS HOGARES COMO EMPLEADORES,98 ACTIV. HOGARES COMO PRODUC. BIENES Y SERV,99 ACTIV. ORGANIZACIONES Y ORGANISMOS EXTRATER...
1993,NaN,NaN,NaN,NaN,NaN,NaN,235.6,NaN,NaN,NaN,...,NaN,NaN,NaN,1.8,NaN,NaN,88.35,NaN,NaN,NaN
1994,267.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995,210.97,NaN,36.06,NaN,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,NaN,180.3,NaN,NaN,NaN,NaN,NaN,NaN
1996,120.2,0.12,NaN,NaN,NaN,NaN,0,NaN,7.21,NaN,...,NaN,NaN,NaN,NaN,NaN,248.71,NaN,NaN,NaN,NaN
1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.84,NaN,...,NaN,NaN,2.4,1.5,NaN,141.09,NaN,NaN,NaN,NaN
1998,1514.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,456.52,NaN,NaN,NaN,NaN,NaN,NaN
1999,1.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.7,296.91,...,NaN,NaN,0,NaN,NaN,NaN,30,NaN,NaN,NaN
2000,25499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,164,NaN,NaN,NaN,NaN,NaN,NaN
2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.19,...,0.6,NaN,NaN,14.86,NaN,NaN,NaN,NaN,NaN,NaN
